In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import MultiComparison
pd.set_option('display.width', 2000)
from utils import readImage
import numpy as np

In [ ]:

nucleus_df = pd.read_csv('csv/nuclei.csv')

regions_df = nucleus_df[(nucleus_df['Location'] != 'Undefined')]
print(regions_df)

In [ ]:
fluo_df = regions_df.groupby(['ImageName', 'Condition', 'Location'])['Ch1Intensity'].mean().reset_index()
pd.set_option('display.max_colwidth', None)

print(fluo_df)
palette = sns.color_palette(['royalblue'], 2)
boxplot = sns.boxplot(data=fluo_df, x='Location', y='Ch1Intensity', hue='Condition', showfliers = False, hue_order=['Sham', 'Contra', 'Ipsi'], palette='Set3')
sns.stripplot(data=fluo_df, x='Location', y='Ch1Intensity', hue='Condition', hue_order=['Sham', 'Contra', 'Ipsi'], dodge=True, palette=palette, alpha=0.7)
handles, labels = boxplot.get_legend_handles_labels()

# Create custom legend
plt.legend(handles=handles[:3], labels=labels[:3])
plt.ylim(0,65)
plt.ylabel('NeuN Fluorescence')
plt.savefig('neuN_fluo_by_region.pdf')
plt.show()

In [ ]:
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

for region in fluo_df['Location'].unique():
    df = fluo_df[fluo_df['Location'] == region]
    print(region)
    anova_results = f_oneway(df[df['Condition'] == 'Contra']['Ch1Intensity'],
                            df[df['Condition'] == 'Ipsi']['Ch1Intensity'],
                            df[df['Condition'] == 'Sham']['Ch1Intensity'])

    # Perform Tukey HSD post hoc test
    tukey_results = pairwise_tukeyhsd(df['Ch1Intensity'], df['Condition'])

    print("ANOVA Results:")
    print("F-statistic:", anova_results.statistic)
    print("p-value:", anova_results.pvalue)

    print("\nTukey HSD Results:")
    print(tukey_results)

In [ ]:

# Now, you can group by the specified columns and count the occurrences
neuron_counts = regions_df.groupby(['Location', 'Condition', 'ImageName', 'CellType']).size().reset_index(name='Nuclei')

In [ ]:
locations = neuron_counts['Location'].unique()

# Loop through each location
for location in locations:
    # Filter dataframe for the current location
    subset_df = neuron_counts[neuron_counts['Location'] == location]
    x_order = ['Undefined', 'neunPositiveLow', 'neunPositive']
    hue_order = ['Sham', 'Contra', 'Ipsi']
    
    # Create the boxplot
    plt.figure(figsize=(10, 6))  # Adjust size if needed
    sns.boxplot(data=subset_df, x='CellType', y='Nuclei', hue='Condition', hue_order=hue_order, order=x_order, palette='Set3')
    
    # Set labels and title
    plt.xlabel('Cell Type')
    plt.ylabel('Nuclei')
    plt.title(f'Boxplot of Nuclei Counts by Cell Type and Condition for Location {location}')
    
    # Rotate x-axis labels for better visibility if needed
    plt.xticks(rotation=45)
    
    # Show the plot
    plt.legend(title='Condition')  # Add legend title
    plt.tight_layout()  # Adjust layout to prevent clipping of labels
    plt.show()

In [ ]:
image_df = pd.read_csv('csv/images.csv')
merged_df = pd.merge(neuron_counts, image_df, on='ImageName')

# Calculate neuron density for each region
merged_df['NeuronDensity'] = merged_df['Nuclei']  # Initialize the column with Nuclei counts
merged_df.loc[merged_df['Location'] == 'CA1', 'NeuronDensity'] /= merged_df['CA1Volume']  # Divide by CA1Volume where Location is CA1
merged_df.loc[merged_df['Location'] == 'CA3', 'NeuronDensity'] /= merged_df['CA3Volume']  # Divide by CA3Volume where Location is CA3
merged_df.loc[merged_df['Location'] == 'DG', 'NeuronDensity'] /= merged_df['DGVolume']    # Divide by DGVolume where Location is DG
merged_df['NeuronDensity'] *= 10**6
# If you want to round the density to a specific number of decimal places
# merged_df['NeuronDensity'] = merged_df['NeuronDensity'].round(decimals=2)

# Display the resulting dataframe
print(merged_df)


In [ ]:
#print(merged_df[(merged_df['CellType'] != 'Undefined') & (merged_df['Location'] == 'DG')])

table_df = merged_df.groupby(['Location', 'Condition_x', 'CellType']).agg({
    'NeuronDensity': ['mean', 'std']
})
print(table_df)

In [ ]:
palette = sns.color_palette(['royalblue'], 2)
for location in locations:
    # Filter dataframe for the current location
    subset_df = merged_df[merged_df['Location'] == location]
    x_order = ['Undefined', 'neunPositiveLow', 'neunPositive']
    hue_order = ['Sham', 'Contra', 'Ipsi']
    
    # Create the boxplot
    plt.figure(figsize=(8, 6))  # Adjust size if needed
    boxplot = sns.boxplot(data=subset_df, x='CellType', y='NeuronDensity', showfliers=False, hue='Condition_x', hue_order=hue_order, order=x_order, palette='Set3')
    stripplot = sns.stripplot(data=subset_df, x='CellType', y='NeuronDensity',order = x_order, hue='Condition_x', hue_order=hue_order, dodge=True, palette=palette, alpha=0.7)
    # Set labels and title
    plt.xlabel('')
    plt.ylabel('Nuclei / 100 \u03bcm^3')
    #plt.title(location)
    
    # Rotate x-axis labels for better visibility if needed
    #plt.xticks(rotation=45)
    
    handles, labels = boxplot.get_legend_handles_labels()

    # Create custom legend
    plt.legend(handles=handles[:3], labels=labels[:3])
    plt.tight_layout()  # Adjust layout to prevent clipping of labels
    name = f"plots/{location}_neurons_volume_v2.pdf"
    #plt.savefig(name)
    plt.show()

In [ ]:
grouped_df = nucleus_df.groupby(['ImageName', 'Condition', 'CellType'])['Ch1Intensity'].median().reset_index()
print(grouped_df)

plt.figure(figsize=(8, 6))
sns.boxplot(data=grouped_df, x= 'CellType', y='Ch1Intensity', hue='Condition',showfliers=False, hue_order=hue_order, order=x_order, palette='Set3')
sns.stripplot(data=grouped_df, x= 'CellType', y='Ch1Intensity', hue='Condition', hue_order=hue_order, order=x_order, dodge=True, palette=palette, alpha=0.7)
plt.xlabel('')
plt.ylabel('NeuN fluorescence')
plt.legend().set_visible(False)
plt.tight_layout()
#plt.savefig('plots/neuN_median_clusters.pdf')
plt.show()


In [ ]:
img_name = 'Sham 1 Contralateral Mouse 6 Slide15 G4green NeuNpink CD86red 40x 5x4 technical replica 2.lsm' # 'Enter name of image as it appears in nucleus_df'
img_path = 'D:/Users\Jonas/nuclei\imagesAndMasks\images_sham\Sham 1 Contralateral Mouse 6 Slide15 G4green NeuNpink CD86red 40x 5x4 technical replica 2.lsm' # 'Path to same image'

neu_df = nucleus_df[nucleus_df['CellType'] == 'neunPositive']
neuLow_df = nucleus_df[nucleus_df['CellType'] == 'neunPositiveLow']
undef_df = nucleus_df[nucleus_df['CellType'] == 'Undefined']
img_neu_df = neu_df[neu_df['ImageName'] == img_name]
img_neuLow_df = neuLow_df[neuLow_df['ImageName'] == img_name]
img_undef_df = undef_df[undef_df['ImageName'] == img_name]

centroids_neu = img_neu_df['Centroid'].apply(eval)
centroids_neuLow = img_neuLow_df['Centroid'].apply(eval)
centroids_undef = img_undef_df['Centroid'].apply(eval)

# Convert centroids to a NumPy array
centroids_neu_array = np.array(centroids_neu.tolist())  # Transpose the array to get the desired shape
centroids_neuLow_array = np.array(centroids_neuLow.tolist())
centroids_undef_array = np.array(centroids_undef.tolist())

from skimage import io
import napari
img= readImage(img_path)

viewer = napari.view_image(img, scale =([0.9278, 0.3459, 0.3459]), channel_axis=3 )
viewer.add_points(centroids_neu_array, size=10, symbol='cross', edge_color='blue', face_color='blue', name='NeuN positive', scale=([0.9278, 0.3459, 0.3459]))
viewer.add_points(centroids_neuLow_array, size=10, symbol='cross', edge_color='green', face_color='green', name='NeuN Low', scale=([0.9278, 0.3459, 0.3459]))
viewer.add_points(centroids_undef_array, size=10, symbol='cross', edge_color='yellow', face_color='yellow', name='NeuN negative', scale=([0.9278, 0.3459, 0.3459]))
napari.run()

In [ ]:
from scipy.stats import f_oneway
from scipy.stats import friedmanchisquare

anova_results = {}
tukey_results = {}

for region in merged_df['Location'].unique():
    for cell_type in merged_df['CellType'].unique():
        type_data = [merged_df[(merged_df['CellType'] == cell_type) & (merged_df['Condition_x'] == condition) & (merged_df['Location'] == region)]['NeuronDensity'] for condition in merged_df['Condition_x'].unique()]
        anova_results[cell_type] = f_oneway(*type_data)
        
        print(f"ANOVA results for {cell_type} in {region}: {anova_results[cell_type]}")
        
        # Perform Tukey's HSD test for each class of neuron
        mc = MultiComparison(merged_df[(merged_df['CellType'] == cell_type) & (merged_df['Location'] == region)]['NeuronDensity'], merged_df[(merged_df['CellType'] == cell_type) & (merged_df['Location'] == region)]['Condition_x'])
        tukey_results[cell_type] = mc.tukeyhsd()
        print(f"Tukey's HSD results for {cell_type} in {region}:\n{tukey_results[cell_type]}")



In [ ]:
undefined_df = nucleus_df[(nucleus_df['CellType'] == 'Undefined')]
print(undefined_df)

In [ ]:
non_neuron_counts = undefined_df.groupby(['Condition', 'ImageName']).size().reset_index(name='NonNuclei')
print(non_neuron_counts)

In [ ]:
x_order = ['Sham', 'Contra', 'Ipsi']
plt.figure(figsize=(8, 6))  # Adjust size if needed
sns.boxplot(data=non_neuron_counts, x='Condition', y='NonNuclei', order=x_order, palette='Set3')
#plt.savefig("plots/nonNeuronsWholeImage.pdf")
plt.show()

In [ ]:
sham_data = non_neuron_counts[non_neuron_counts['Condition'] == 'Sham']['NonNuclei']
contra_data = non_neuron_counts[non_neuron_counts['Condition'] == 'Contra']['NonNuclei']
ipsi_data = non_neuron_counts[non_neuron_counts['Condition'] == 'Ipsi']['NonNuclei']

# Perform ANOVA
f_statistic, p_value = f_oneway(sham_data, contra_data, ipsi_data)

print("F-statistic:", f_statistic)
print("p-value:", p_value)

In [ ]:
undefined_df2 = undefined_df[undefined_df['Location'] != 'Undefined']

non_neuron_counts = undefined_df2.groupby(['Condition', 'ImageName']).size().reset_index(name='NonNeurons')
print(non_neuron_counts)

In [ ]:
plt.figure(figsize=(8, 6))  # Adjust size if needed
sns.boxplot(data=non_neuron_counts, x='Condition', y='NonNeurons', order=x_order, palette='Set3')
#plt.savefig("plots/nonNeuronsInROIS.pdf")
plt.show()

In [ ]:
sham_data = non_neuron_counts[non_neuron_counts['Condition'] == 'Sham']['NonNuclei']
contra_data = non_neuron_counts[non_neuron_counts['Condition'] == 'Contra']['NonNuclei']
ipsi_data = non_neuron_counts[non_neuron_counts['Condition'] == 'Ipsi']['NonNuclei']

# Perform ANOVA
f_statistic, p_value = f_oneway(sham_data, contra_data, ipsi_data)

print("F-statistic:", f_statistic)
print("p-value:", p_value)